In [179]:
import pandas as pd

In [180]:
xl = pd.ExcelFile('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\summary_all_data.xlsx')

xl.sheet_names  # see all sheet names

['percent GDP', 'GDP']

In [181]:
GDP = xl.parse("GDP")  # read a specific sheet to DataFrame

In [182]:
GDP2 = GDP.set_index(["wbregionname","year","ssp"]).gdp.unstack("wbregionname")

In [183]:
GDP2.columns

Index(['East Asia and Pacific', 'Europe and Central Asia',
       'Latin America and Caribbean', 'Middle East and North Africa',
       'South Asia', 'Sub-Saharan Africa'],
      dtype='object', name='wbregionname')

In [184]:
GDP2 = GDP2.rename(columns={"Europe and Central Asia":"Eastern Europe and Central Asia",
                            "Middle East and North Africa":"Middle-East and North Africa"})

In [185]:
GDP2["Africa"] = GDP2["Middle-East and North Africa"]+GDP2["Sub-Saharan Africa"]
GDP2["Asia"] = GDP2["East Asia and Pacific"]+GDP2["South Asia"]
GDP2["all_countries"] = GDP2["Africa"]+GDP2["Asia"]+GDP2["Latin America and Caribbean"]+GDP2["Eastern Europe and Central Asia"]

In [186]:
GDP2.stack().sample(5)

year  ssp   wbregionname                   
2018  ssp5  Eastern Europe and Central Asia    3.656326e+12
2028  ssp1  Africa                             9.273437e+12
2027  ssp4  Latin America and Caribbean        1.180309e+13
2026  ssp4  Asia                               3.238376e+13
2027  ssp1  all_countries                      6.392547e+13
dtype: float64

In [187]:
res = xl.parse("percent GDP")

In [188]:
res["sector"] = res.sector.replace({"WASH":"wash"})

In [189]:
res.region.unique()

array(['all_countries', 'Africa', 'Asia', 'Latin America and Caribbean',
       'Former soviet Union', 'Middle-East and North Africa',
       'Sub-Saharan Africa', 'South Asia', 'East Asia and Pacific',
       'Eastern Europe and Central Asia'], dtype=object)

In [190]:
res["region"] = res.region.replace({'Former soviet Union':'Eastern Europe and Central Asia'})

In [191]:
res.loc[res.sector=="energy","ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="transport")&(res.scenario=="max"),"ssp"] = "ssp3"
res.loc[(res.sector=="wash")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="max"),"ssp"] = "ssp4"
res.loc[(res.sector=="coastal protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="max"),"ssp"] = "ssp5"
res.loc[(res.sector=="river protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="river protection")&(res.scenario=="min"),"ssp"] = "ssp3"
res.loc[(res.sector=="river protection")&(res.scenario=="max"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="irrigation")&(res.scenario=="max"),"ssp"] = "ssp2"

res.loc[(res.region=="all_countries"),"ssp"] = "ssp2"

In [192]:
res[res.ssp.isnull()]

,region,sector,scenario,variable,capital vs maintenance,data,ssp


In [193]:
mydata = res.merge(GDP2.stack().reset_index().rename(columns={"wbregionname":"region",0:"GDP"}),on=["region","ssp"])

In [194]:
mydata["cost_bUSD"] = 1e-9/100*mydata.data*mydata.GDP

In [195]:
d = 0.06
mydata["discount_factor"] = (1+d)**(1-(mydata.year.astype(int)-2015))

In [196]:
mydata["cost_bUSD_disc"] = mydata["cost_bUSD"]*mydata["discount_factor"]

In [197]:
newdata = mydata.set_index(["region","sector","scenario","capital vs maintenance","year"]).cost_bUSD_disc.sum(level=["region","sector","scenario","capital vs maintenance"])

In [198]:
newdata2 = newdata.reset_index()

In [199]:
newdata2["cost_bUSD_disc_annual"] = newdata2["cost_bUSD_disc"]/15

In [224]:
totals = pd.DataFrame()

# Energy

In [200]:
energy = newdata2.loc[(newdata2.sector=="energy")&(newdata2["capital vs maintenance"]=="capital"),:]

## keep for overview

In [228]:
energy.loc[energy.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario").reset_index()

,scenario,cost_bUSD_disc_annual
0,best,778.114139
1,min,298.587895
2,max,1020.534268


In [229]:
temp = energy.loc[energy.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario").reset_index()
temp["sector"] = "energy"

In [232]:
totals = temp

In [202]:
energy.loc[energy.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
0,min,317.687083
5,best,776.568426
10,max,1058.956944


# Transport

In [203]:
transport = newdata2.loc[(newdata2.sector=="transport")&(newdata2["capital vs maintenance"]=="capital"),:]

## keep for overview

In [204]:
transport.loc[transport.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
best     416.599217
min      156.796355
max     1060.207794
Name: cost_bUSD_disc_annual, dtype: float64

In [233]:
temp = transport.loc[transport.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario").reset_index()
temp["sector"] = "transport"

In [235]:
totals = totals.append(temp)

In [205]:
transport.loc[transport.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
1,min,187.082394
6,best,458.881343
11,max,1164.852639


# WASH

In [206]:
wash = newdata2.loc[(newdata2.sector=="wash")&(newdata2["capital vs maintenance"]=="capital"),:]

## keep for overview

In [207]:
wash.loc[wash.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
best    198.148016
min     115.630826
max     228.925546
Name: cost_bUSD_disc_annual, dtype: float64

In [236]:
temp = wash.loc[wash.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario").reset_index()
temp["sector"] = "wash"

In [237]:
totals = totals.append(temp)

In [208]:
wash.loc[wash.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
2,min,112.955407
7,best,194.142106
12,max,229.440671


# Flood

In [209]:
coastal_flood = newdata2.loc[(newdata2.sector=="coastal protection")&(newdata2["capital vs maintenance"]=="capital"),:]

In [210]:
coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
best    42.919258
min      2.349903
max     56.573056
Name: cost_bUSD_disc_annual, dtype: float64

In [211]:
river_flood = newdata2.loc[(newdata2.sector=="river protection")&(newdata2["capital vs maintenance"]=="capital"),:]

In [212]:
river_flood.loc[river_flood.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
best     60.137460
max     278.575892
min      20.589857
Name: cost_bUSD_disc_annual, dtype: float64

In [213]:
flood = coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")+\
river_flood.loc[river_flood.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

## keep for overview

In [214]:
flood

scenario
best    103.056717
max     335.148948
min      22.939760
Name: cost_bUSD_disc_annual, dtype: float64

In [238]:
temp = flood.reset_index()
temp["sector"] = "flood"

In [239]:
totals = totals.append(temp)

In [215]:
newdata2.loc[(newdata2.sector=="flood")&(newdata2["capital vs maintenance"]=="capital"),:]

,region,sector,scenario,capital vs maintenance,cost_bUSD_disc,cost_bUSD_disc_annual
3,all_countries,flood,min,capital,317.687083,21.179139
8,all_countries,flood,best,capital,1694.331111,112.955407
13,all_countries,flood,max,capital,5294.784722,352.985648


# Irrigation

In [216]:
irrigation = newdata2.loc[(newdata2.sector=="irrigation")&(newdata2["capital vs maintenance"]=="capital"),:]

## keep for overview

In [217]:
irrigation.loc[irrigation.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
best     49.619364
max     100.434281
min      43.592279
Name: cost_bUSD_disc_annual, dtype: float64

In [240]:
temp = irrigation.loc[irrigation.region!="all_countries",:].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario").reset_index()
temp["sector"] = "irrigation"

In [241]:
totals = totals.append(temp)

In [218]:
irrigation.loc[irrigation.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
4,min,42.358278
9,best,45.888134
14,max,70.597130


In [219]:
newdata2[(newdata2.region!="all_countries")&(newdata2["capital vs maintenance"]=="capital")].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
best    1545.537453
min      637.547114
max     2745.250838
Name: cost_bUSD_disc_annual, dtype: float64

In [220]:
newdata2[(newdata2.region=="all_countries")&(newdata2["capital vs maintenance"]=="capital")].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
min      681.262301
best    1588.435417
max     2876.833032
Name: cost_bUSD_disc_annual, dtype: float64

In [242]:
totals

,scenario,cost_bUSD_disc_annual,sector
0,best,778.114139,energy
1,min,298.587895,energy
2,max,1020.534268,energy
0,best,416.599217,transport
1,min,156.796355,transport
2,max,1060.207794,transport
0,best,198.148016,wash
1,min,115.630826,wash
2,max,228.925546,wash
0,best,103.056717,flood


In [243]:
totals.set_index(["sector","scenario"]).sum(level="scenario")

,cost_bUSD_disc_annual
scenario,
best,1545.537453
min,637.547114
max,2745.250838


In [250]:
totals[totals.sector=="energy"]

,scenario,cost_bUSD_disc_annual,sector
0,best,778.114139,energy
1,min,298.587895,energy
2,max,1020.534268,energy
